# Prop-on-prop Surrogate Model

## Analysis Functions

In [1]:
# Singular kernel
zeta_sing(r) = r==0 ? 1 : 0

# erf Gaussian kernel
zeta_gauserf(r) = 1/(2*pi)^(3/2) * exp(-r^2/2)

# Gaussian kernel
zeta_gaus(r) = 3/(4*pi)*exp(-r^3)

# Winckelmans algebraic kernel
zeta_wnklmns(r) = 7.5 / (4*pi) / (r^2 + 1)^3.5

zeta_wnklmns (generic function with 1 method)

In [2]:
using Statistics
using LinearAlgebra

# Returns a radial basis function interpolation of a field
# with values `val` at positions `Xp`. `zeta` is the chosen
# basis function
function generate_RBF(Xp, val, zeta; sigmas=0.1)
    
    # ERROR CASES
    if size(Xp,1)!=size(val,1)
        error("size(Xp,1)!=size(val,1)")
    end
    
    Np = size(Xp, 1)                     # Number of data points
    
    if size(sigmas)==()
        sgms = sigmas*ones(Np)           # Spreading of every basis function
    else
        sgms = sigmas
    end
        
    # j-th scaled basis evaluated at X
    zetasgm(j, X) = zeta(Statistics.norm(X-Xp[j])/sgms[j])/sgms[j]^3
     
    # Matrix with basis functions evaluated at every point
    # Z[i,j] corresponds to the j-th basis evaluated at i-th point
    Z = [zetasgm(j, Xi) for Xi in Xp, j in 1:Np]
    
    # Solves for the alpha coefficients of every basis
#     A = Z\val
    A = pinv(Z)*val
    
    # Generates RBF interpolation function
    rbf(X) = sum([A[j]*zetasgm(j, X) for j in 1:Np])
    
    return rbf, A
end

generate_RBF (generic function with 1 method)

## Open Database

In [3]:
using Pkg
Pkg.add("JuliaDB")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [4]:
# using Plotly
Pkg.add("LaTeXStrings")
Pkg.add("Interact")
Pkg.add("Plots")
using LaTeXStrings
using Interact
import JuliaDB
# using StatPlots
# stplscatter = StatPlots.scatter
using Plots
db = JuliaDB

data = db.loadtable("data/sweep_apc10x7_20181126.csv")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/614413f37706119d59ac1ae6e9e38037cd09365b-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/614413f37706119d59ac1ae6e9e38037cd09365b-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /Users/aschenk/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1190


Table with 638 rows, 33 columns:
Columns:
#   colname         type
───────────────────────────────────────────
1   rotors          Int64
2   rotor           Int64
3   nrevs_ave       Int64
4   nrevs_tot       Int64
5   corrotating     String
6   doR             Float64
7   J               Float64
8   ReD07           Float64
9   CT_mean         Float64
10  CT_std          Float64
11  CQ_mean         Float64
12  CQ_std          Float64
13  eta_mean        Float64
14  eta_std         Float64
15  nelements       Int64
16  nsteps_per_rev  Int64
17  xfoil           String
18  sigmafactor     Float64
19  conv_file       String
20  args_file       String
21  doR_ref         Float64
22  CT_mean_ref     Float64
23  CT_std_ref      Float64
24  CQ_mean_ref     Float64
25  CQ_std_ref      Float64
26  eta_mean_ref    Float64
27  eta_std_ref     Float64
28  normCT_mean     Float64
29  normCT_std      Float64
30  normCQ_mean     Float64
31  normCQ_std      Float64
32  normeta_mean    Union{Missing, Fl

## Surrogate Model

In [5]:
# Variables of the problem
vars = [:J, :doR, :ReD07]

# Scaling of every variable
scl = Dict()
for var in vars
    valmax = reduce(max, data; select=var)
    valmin = reduce(min, data; select=var)
    scl[var] = [valmin, valmax]
end    

# GUI-enabled plot
asd = plotly()

# Axes to plot
xaxis = :J
yaxis = :doR
zaxiss = [:normCT_mean, :normCQ_mean, :normeta_mean, 
            :normCT_std, :normCQ_std, :normeta_std, 
            :CT_mean, :CQ_mean, :eta_mean, 
            :CT_std, :CQ_std, :eta_std]

axis_labels= Dict(
#     :J => L"Advance ratio $J$",
#     :doR => L"Separation distance $\delta$",
#     :ReD07 => L"Reynolds number $\mathrm{Re}$",
#     :normCT_mean => L"Normalized thrust $C_{T,\mu}$",
#     :normCQ_mean => L"Normalized torque $C_{Q,\mu}$",
#     :normCT_std => L"Normalized thrust fluctuation $C_{T,\sigma}$",
#     :normCQ_std => L"Normalized torque fluctuation $C_{T,\sigma}$",
    :J => "Advance ratio J",
    :doR => "Separation δ",
    :ReD07 => "Reynolds number",
    :normCT_mean => "Thrust CT",
    :normCQ_mean => "Torque CQ",
    :normCT_std => "Thrust fluctuation CT",
    :normCQ_std => "Torque fluctuation CT",
    )



# Standard deviations associated to every zaxis (std, normalize)
stdzaxiss = Dict(   :normCT_mean => (:CT_std, (:CT_mean, :CT_mean_ref, :CT_std_ref)),
                    :normCQ_mean => (:CQ_std, (:CQ_mean, :CQ_mean_ref, :CQ_std_ref)),
                    :normeta_mean => (:eta_std, (:eta_mean, :eta_mean_ref, :eta_std_ref)),
                    :CT_mean => (:CT_std, nothing),
                    :CQ_mean => (:CQ_std, nothing),
                    :eta_mean => (:eta_std, nothing))

# Range that considers as negligible interaction
negl_rng = 0.0025

global first_flag = true

# Interactive plot
mp = @manipulate for fluctX1e3 in [false, true],
                     zeta in [zeta_gauserf, zeta_gaus, zeta_wnklmns, zeta_sing],
                     zmin in dropdown(0:0.025:1, value=1, label="zmin"),
                     zmax in dropdown(1:0.05:20, value=1, label="zmax"),
                     sigma in dropdown(0.25:0.25:15.0, value=8.0, label="Smoothing radius"),
                     plot_surface in [true, false],
                     actwireframe in [false, true],
                     rotor in [1,2],
                     corrotating in [false, true],
                     Re in [1e5, 5e5, 1e6, 1.5e6], 
                     zaxis in zaxiss
    
    global first_flag = true
    
    xs = []
    ys = []
    zs = []
    
    validval(x) = typeof(x) != Missing && x != nothing && !isnan(x)
    
    function fcritfav(x)
        if validval(getfield(x, zaxis)) 
            return getfield(x, zaxis)>1+negl_rng 
        else 
            false 
        end
    end
    function fcritnegl(x)
        if validval(getfield(x, zaxis)) 
            return 1-negl_rng<=getfield(x, zaxis)<=1+negl_rng
        else 
            false 
        end
    end
    function fcritnega(x)
        if validval(getfield(x, zaxis)) 
            return 1-negl_rng>getfield(x, zaxis)
        else 
            false 
        end
    end
    
    plotobj = nothing
    
    
    for (lbl, fcrit, clr) in [# Splits the data in different criteria
        # Favorable interaction
        ("Favorable", fcritfav, :green),
        # Negligible interaction
        ("Negligible", fcritnegl, :blue),
        # Negative interaction
        ("Negative", fcritnega, :red) ,
                            ]
    
        # Fetches the data filtered by criterion
        fldata = db.filter(i -> (i.corrotating=="$corrotating" && i.ReD07==Re
                                && i.doR_ref>=2.0 
                                && fcrit(i) 
                                && i.rotor==rotor
                                ), data)
        
        # Data to plot
        this_xs = db.select(fldata, xaxis)
        this_ys = db.select(fldata, yaxis)
        this_zs = db.select(fldata, zaxis)
#         this_zs = [z.value for z in this_zs]
            
        # Selects plot function to use
        if first_flag
            plotfun = Plots.scatter
            global first_flag = false
        else
            plotfun = Plots.scatter!
        end
        
        optargs = zmin==zmax==1.0 ? [] : [(:zlims, (zmin, zmax))]
        
        plotobj = plotfun(this_xs, this_ys, this_zs,
                xlabel=xaxis in keys(axis_labels) ? axis_labels[xaxis] : string(xaxis),
                ylabel=yaxis in keys(axis_labels) ? axis_labels[yaxis] : string(yaxis),
                zlabel=zaxis in keys(axis_labels) ? axis_labels[zaxis] : string(zaxis),
                        markerstrokealpha = 1.0, label=lbl,
                        markersize=2, markeralpha = 1.0, color=clr,
                        ; optargs...
        )
        
        # Error bars
        if fluctX1e3 && zaxis in keys(stdzaxiss)
            std, nrm = stdzaxiss[zaxis]
            if nrm!=nothing
                stds = 0.001*(db.select(fldata, std)./db.select(fldata, nrm[3])
                            )./(db.select(fldata, nrm[1])./db.select(fldata, nrm[2]))
            else
                stds = db.select(fldata, std)
            end
            Plots.plot!([x*ones(2) for x in this_xs],
                        [y*ones(2) for y in this_ys],
                        [z .+ stds[i]*[1,-1] for (i,z) in 
                                    enumerate(this_zs)], 
                        color=:black, label=" ", alpha=0.25)
        end
        
        xs = vcat(xs, this_xs)
        ys = vcat(ys, this_ys)
        zs = vcat(zs, this_zs)
    end
    
    # Position of every datapoint in this plot (only J and doR are variables)
    Np = size(xs, 1)          # Number of datapoints
#     Xp = [[xs[i], ys[i]] for i in 1:Np]
#     vals = [z for z in zs]
    # Scaling is done here
    xmn, xmx = scl[xaxis]
    ymn, ymx = scl[yaxis]
    zmn = reduce(min, data; select=zaxis)
    zmx = reduce(max, data; select=zaxis)
    Xp = [[(xs[i]-xmn)/(xmx-xmn), 
           (ys[i]-ymn)/(ymx-ymn)] for i in 1:Np]
    vals = [(z-zmn)/(zmx-zmn) for z in zs]
    
#     Generates radial basis function interpolation
    rbf_raw, A = generate_RBF(Xp, vals, zeta; sigmas=sigma)
    
    # Wrapes the scaled rbf
    rbf(X) = rbf_raw([(X[1]-xmn)/(xmx-xmn), 
                      (X[2]-ymn)/(ymx-ymn)])*(zmx-zmn) + zmn
    
    if plot_surface
        # Surface plot
        n = 100
        surfx = collect(range(scl[xaxis][1], stop=scl[xaxis][2], length=n))
        surfy = collect(range(scl[yaxis][1], stop=scl[yaxis][2], length=n))
        surfz = [rbf([x,y]) for y in surfy, x in surfx]
        Plots.surface!(surfx, surfy, surfz, alpha=0.8)
    #     Plots.wireframe!(surfx, surfy, surfz, alpha=1.0, linewidth=10, c=:blue)

        if actwireframe
            # Wireframe
    #         for (i,x) in enumerate(surfx)
    #              Plots.plot!([x for val in surfx], surfy, surfz[i, :], 
    #                             label="", c=:black, alpha=0.25)
    #         end
            for (i,y) in enumerate(surfy)
                 Plots.plot!(surfx, [y for val in surfy], surfz[i, :], 
                                label="", c=:black, alpha=0.25)
            end
        end
    end
    
#     # Calculates RBF error
#     rbf_zs = [rbf([xs[i], ys[i]]) for i in 1:size(zs, 1)]
#     err = mean(abs.(zs-rbf_zs)./zs)
#     title!("RBF error = $(round(err*100,3))%")
    
    # This is is so that the plot is the last thing it displays
#     Plots.plot!([], [], [])
    plotobj
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope("knockout-component-f8917bb9-1060-4512-928f-8df9d436087a", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["fluctX1e3"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :span), Any[Node{DOM}(DOM(:html, :label), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text : key")), 0)], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget button ' : true}")), 1)], Dict{Symbol,Any}(:attributes=>("data-bind"=>"foreach : options_js"),:className=>"buttons has-addons is-centered"), 2)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 3), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM